In [1]:
import pandas as pd
import numpy as np

In [ ]:
headlines = pd.read_csv('/raw_partner_headlines.csv')

headlines.head()

,Unnamed: 0,headline,url,publisher,date,stock
0,2,Agilent Technologies Announces Pricing of $5……...,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01 00:00:00,A
1,3,Agilent (A) Gears Up for Q2 Earnings: What's i...,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18 00:00:00,A
2,4,J.P. Morgan Asset Management Announces Liquida...,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15 00:00:00,A
3,5,"Pershing Square Capital Management, L.P. Buys ...",http://www.gurufocus.com/news/1138704/pershing...,GuruFocus,2020-05-15 00:00:00,A
4,6,Agilent Awards Trilogy Sciences with a Golden ...,http://www.gurufocus.com/news/1134012/agilent-...,GuruFocus,2020-05-12 00:00:00,A


In [6]:
import sys
import boto3
from botocore.config import Config
import pandas as pd
import json
import time
import os
import numpy as np
import pyarrow
import traceback
from langchain.embeddings import BedrockEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import BedrockChat
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import BedrockEmbeddings
from langchain.embeddings import SpacyEmbeddings
from dotenv import load_dotenv
from typing import Optional


# spaCy

In [ ]:
# spaCy
def generate_spacy_vector_embedding(text):
    embedder = SpacyEmbeddings(model_name="en_core_web_md")
    query_embedding = embedder.embed_query(text)

    return(np.array(query_embedding))

In [8]:
# Mathematical formula for cosine similarity
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity

In [12]:
# Subset data first 5 and last 5 rows
sample = pd.concat([headlines.head(5), headlines.tail(5)]).copy()
sample.reset_index(drop=True, inplace=True)

sample

,Unnamed: 0,headline,url,publisher,date,stock
0,2,Agilent Technologies Announces Pricing of $5……...,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01 00:00:00,A
1,3,Agilent (A) Gears Up for Q2 Earnings: What's i...,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18 00:00:00,A
2,4,J.P. Morgan Asset Management Announces Liquida...,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15 00:00:00,A
3,5,"Pershing Square Capital Management, L.P. Buys ...",http://www.gurufocus.com/news/1138704/pershing...,GuruFocus,2020-05-15 00:00:00,A
4,6,Agilent Awards Trilogy Sciences with a Golden ...,http://www.gurufocus.com/news/1134012/agilent-...,GuruFocus,2020-05-12 00:00:00,A
5,1849874,Consumer Cyclical Sector Wrap,https://www.benzinga.com/content/12/08/2846030...,webmaster,2012-08-20 00:00:00,ZX
6,1849875,Consumer Cyclical Sector Wrap,https://www.benzinga.com/content/12/07/2767124...,webmaster,2012-07-23 00:00:00,ZX
7,1849876,Zacks #5 Rank Additions for Monday - Tale of t...,http://www.zacks.com/stock/news/73497/here-are...,Zacks,2012-04-23 00:00:00,ZX
8,1849877,4 Stock Strategies From Wall Street: Feb. 9 (U...,http://www.thestreet.com/story/11409053/1/4-st...,TheStreet.Com,2012-02-09 00:00:00,ZX
9,1849878,4 Stock Strategies From Wall Street: Feb. 9,https://www.benzinga.com/content/thestreet-com...,webmaster,2012-02-09 00:00:00,ZX


In [13]:
sample['embedding'] = sample['headline'].apply(generate_spacy_vector_embedding)

sample

,Unnamed: 0,headline,url,publisher,date,stock,embedding
0,2,Agilent Technologies Announces Pricing of $5……...,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01 00:00:00,A,"[-0.6346091628074646, 0.2849180996417999, 0.20..."
1,3,Agilent (A) Gears Up for Q2 Earnings: What's i...,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18 00:00:00,A,"[-0.700508713722229, 0.21515785157680511, -0.0..."
2,4,J.P. Morgan Asset Management Announces Liquida...,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15 00:00:00,A,"[-0.7054591178894043, 0.24724848568439484, 0.1..."
3,5,"Pershing Square Capital Management, L.P. Buys ...",http://www.gurufocus.com/news/1138704/pershing...,GuruFocus,2020-05-15 00:00:00,A,"[-0.7248147130012512, 0.19851821660995483, 0.0..."
4,6,Agilent Awards Trilogy Sciences with a Golden ...,http://www.gurufocus.com/news/1134012/agilent-...,GuruFocus,2020-05-12 00:00:00,A,"[-0.6144430041313171, 0.1564493030309677, 0.08..."
5,1849874,Consumer Cyclical Sector Wrap,https://www.benzinga.com/content/12/08/2846030...,webmaster,2012-08-20 00:00:00,ZX,"[-0.7226225137710571, 0.34309500455856323, -0...."
6,1849875,Consumer Cyclical Sector Wrap,https://www.benzinga.com/content/12/07/2767124...,webmaster,2012-07-23 00:00:00,ZX,"[-0.7226225137710571, 0.34309500455856323, -0...."
7,1849876,Zacks #5 Rank Additions for Monday - Tale of t...,http://www.zacks.com/stock/news/73497/here-are...,Zacks,2012-04-23 00:00:00,ZX,"[-0.6963266730308533, 0.32234063744544983, 0.1..."
8,1849877,4 Stock Strategies From Wall Street: Feb. 9 (U...,http://www.thestreet.com/story/11409053/1/4-st...,TheStreet.Com,2012-02-09 00:00:00,ZX,"[-0.6737083792686462, 0.380733847618103, 0.131..."
9,1849878,4 Stock Strategies From Wall Street: Feb. 9,https://www.benzinga.com/content/thestreet-com...,webmaster,2012-02-09 00:00:00,ZX,"[-0.6689377427101135, 0.32956668734550476, 0.1..."


In [21]:
# Test headline
test_headline = "Stock markets, Tariff War"

# Generate embedding for the test
query_vec = generate_spacy_vector_embedding(test_headline)

In [22]:
sample['cosine_score'] = sample['embedding'].apply(lambda x: cosine_similarity(x, query_vec))

In [23]:
print(sample[['headline', 'cosine_score']].sort_values(by='cosine_score', ascending=False))

                                            headline  cosine_score
2  J.P. Morgan Asset Management Announces Liquida...      0.725605
3  Pershing Square Capital Management, L.P. Buys ...      0.649390
5                      Consumer Cyclical Sector Wrap      0.645008
6                      Consumer Cyclical Sector Wrap      0.645008
1  Agilent (A) Gears Up for Q2 Earnings: What's i...      0.603486
7  Zacks #5 Rank Additions for Monday - Tale of t...      0.601360
0  Agilent Technologies Announces Pricing of $5……...      0.586759
9        4 Stock Strategies From Wall Street: Feb. 9      0.477287
4  Agilent Awards Trilogy Sciences with a Golden ...      0.443671
8  4 Stock Strategies From Wall Street: Feb. 9 (U...      0.428164


# Amazon Titan

In [ ]:
class TitanEmbeddings(object):
    accept = "application/json"
    content_type = "application/json"
    
    def __init__(self, model_id="amazon.titan-embed-text-v2:0", boto3_client=None, region_name='us-west-1'):
        
        if boto3_client:
            self.bedrock_boto3 = boto3_client
        else:
            # self.bedrock_boto3 = boto3.client(service_name='bedrock-runtime')
            self.bedrock_boto3 = boto3.client(
                service_name='bedrock-runtime', 
                region_name=region_name, 
            )
        self.model_id = model_id

    def __call__(self, text, dimensions, normalize=True):
        """
        Returns Titan Embeddings

        Args:
            text (str): text to embed
            dimensions (int): Number of output dimensions.
            normalize (bool): Whether to return the normalized embedding or not.

        Return:
            List[float]: Embedding
            
        """

        body = json.dumps({
            "inputText": text,
            "dimensions": dimensions,
            "normalize": normalize
        })

        response = self.bedrock_boto3.invoke_model(
            body=body, modelId=self.model_id, accept=self.accept, contentType=self.content_type
        )

        response_body = json.loads(response.get('body').read())

        return response_body['embedding']


In [18]:
def get_bedrock_client(assumed_role: Optional[str] = None, region: Optional[str] = 'us-west-2',runtime: Optional[bool] = True,external_id=None, ep_url=None):
    """Create a boto3 client for Amazon Bedrock, with optional configuration overrides 
    """
    target_region = region

    #print(f"Create new client\n  Using region: {target_region}:external_id={external_id}: ")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        if external_id:
            response = sts.assume_role(
                RoleArn=str(assumed_role),
                RoleSessionName="langchain-llm-1",
                ExternalId=external_id
            )
        else:
            response = sts.assume_role(
                RoleArn=str(assumed_role),
                RoleSessionName="langchain-llm-1",
            )
        print(f"Using role: {assumed_role} ... sts::successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]

    if runtime:
        service_name='bedrock-runtime'
    else:
        service_name='bedrock'

    if ep_url:
        bedrock_client = session.client(service_name=service_name,config=retry_config,endpoint_url = ep_url, **client_kwargs )
    else:
        bedrock_client = session.client(service_name=service_name,config=retry_config, **client_kwargs )

    #print("boto3 Bedrock client successfully created!")
    #print(bedrock_client._endpoint)
    return bedrock_client

In [34]:
def generate_titan_vector_embedding(text, embedding_size):
    aws_client = get_bedrock_client()
    bedrock_embeddings = TitanEmbeddings(model_id="amazon.titan-embed-text-v2:0", boto3_client=aws_client)
    
    modelId = "amazon.titan-embed-text-v2:0"  # 
    accept = "application/json"
    contentType = "application/json"
    

    model_input = {
        "inputText": text  # NOT a list! Just a plain string
    }
    
    body = json.dumps(model_input)
    response = aws_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)    
    response_body = json.loads(response.get('body').read())
    embedding = response_body.get("embedding")

    return np.array(embedding)
   

In [35]:
sample2 = pd.concat([headlines.head(5), headlines.tail(5)]).copy()
sample2.reset_index(drop=True, inplace=True)

sample2

,Unnamed: 0,headline,url,publisher,date,stock
0,2,Agilent Technologies Announces Pricing of $5……...,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01 00:00:00,A
1,3,Agilent (A) Gears Up for Q2 Earnings: What's i...,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18 00:00:00,A
2,4,J.P. Morgan Asset Management Announces Liquida...,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15 00:00:00,A
3,5,"Pershing Square Capital Management, L.P. Buys ...",http://www.gurufocus.com/news/1138704/pershing...,GuruFocus,2020-05-15 00:00:00,A
4,6,Agilent Awards Trilogy Sciences with a Golden ...,http://www.gurufocus.com/news/1134012/agilent-...,GuruFocus,2020-05-12 00:00:00,A
5,1849874,Consumer Cyclical Sector Wrap,https://www.benzinga.com/content/12/08/2846030...,webmaster,2012-08-20 00:00:00,ZX
6,1849875,Consumer Cyclical Sector Wrap,https://www.benzinga.com/content/12/07/2767124...,webmaster,2012-07-23 00:00:00,ZX
7,1849876,Zacks #5 Rank Additions for Monday - Tale of t...,http://www.zacks.com/stock/news/73497/here-are...,Zacks,2012-04-23 00:00:00,ZX
8,1849877,4 Stock Strategies From Wall Street: Feb. 9 (U...,http://www.thestreet.com/story/11409053/1/4-st...,TheStreet.Com,2012-02-09 00:00:00,ZX
9,1849878,4 Stock Strategies From Wall Street: Feb. 9,https://www.benzinga.com/content/thestreet-com...,webmaster,2012-02-09 00:00:00,ZX


In [36]:
sample2['embedding'] = sample2['headline'].apply(generate_titan_vector_embedding, embedding_size=768)

sample2

,Unnamed: 0,headline,url,publisher,date,stock,embedding
0,2,Agilent Technologies Announces Pricing of $5……...,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01 00:00:00,A,"[-0.07962369173765182, 0.07772404700517654, 0...."
1,3,Agilent (A) Gears Up for Q2 Earnings: What's i...,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18 00:00:00,A,"[0.00822487473487854, 0.03508780524134636, 0.0..."
2,4,J.P. Morgan Asset Management Announces Liquida...,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15 00:00:00,A,"[-0.041430726647377014, -0.05488956347107887, ..."
3,5,"Pershing Square Capital Management, L.P. Buys ...",http://www.gurufocus.com/news/1138704/pershing...,GuruFocus,2020-05-15 00:00:00,A,"[-0.07014969736337662, 0.007181856315582991, 0..."
4,6,Agilent Awards Trilogy Sciences with a Golden ...,http://www.gurufocus.com/news/1134012/agilent-...,GuruFocus,2020-05-12 00:00:00,A,"[0.000412996974773705, 0.06498735398054123, 0...."
5,1849874,Consumer Cyclical Sector Wrap,https://www.benzinga.com/content/12/08/2846030...,webmaster,2012-08-20 00:00:00,ZX,"[-0.045372266322374344, -0.026911698281764984,..."
6,1849875,Consumer Cyclical Sector Wrap,https://www.benzinga.com/content/12/07/2767124...,webmaster,2012-07-23 00:00:00,ZX,"[-0.045372266322374344, -0.026911698281764984,..."
7,1849876,Zacks #5 Rank Additions for Monday - Tale of t...,http://www.zacks.com/stock/news/73497/here-are...,Zacks,2012-04-23 00:00:00,ZX,"[-0.01694352552294731, -0.029176775366067886, ..."
8,1849877,4 Stock Strategies From Wall Street: Feb. 9 (U...,http://www.thestreet.com/story/11409053/1/4-st...,TheStreet.Com,2012-02-09 00:00:00,ZX,"[0.0320182666182518, 0.00162030931096524, -0.0..."
9,1849878,4 Stock Strategies From Wall Street: Feb. 9,https://www.benzinga.com/content/thestreet-com...,webmaster,2012-02-09 00:00:00,ZX,"[0.040879957377910614, 0.027427317574620247, -..."


In [39]:
# Test headline
test_headline = "Stock markets, Tariff War"

# Generate embedding for the test
query_vec = generate_titan_vector_embedding(test_headline, embedding_size=768)

In [40]:
sample2['cosine_score'] = sample2['embedding'].apply(lambda x: cosine_similarity(x, query_vec))

print(sample2[['headline', 'cosine_score']].sort_values(by='cosine_score', ascending=False))

                                            headline  cosine_score
6                      Consumer Cyclical Sector Wrap      0.163043
5                      Consumer Cyclical Sector Wrap      0.163043
9        4 Stock Strategies From Wall Street: Feb. 9      0.132033
8  4 Stock Strategies From Wall Street: Feb. 9 (U...      0.107028
1  Agilent (A) Gears Up for Q2 Earnings: What's i...      0.093203
7  Zacks #5 Rank Additions for Monday - Tale of t...      0.090282
4  Agilent Awards Trilogy Sciences with a Golden ...      0.069814
0  Agilent Technologies Announces Pricing of $5……...      0.064041
2  J.P. Morgan Asset Management Announces Liquida...      0.060680
3  Pershing Square Capital Management, L.P. Buys ...      0.052951


# Cohere

In [41]:
# send in an array size of one and only return the 0th element
def generate_cohere_vector_embedding(text_data):
    aws_client = get_bedrock_client()
    input_type = "clustering"
    truncate = "NONE" # optional
    model_id = "cohere.embed-english-v3" # or "cohere.embed-multilingual-v3"
    
    # Create the JSON payload for the request
    json_params = {
            'texts': [text_data],
            'truncate': truncate, 
            "input_type": input_type
        }
    json_body = json.dumps(json_params)
    params = {'body': json_body, 'modelId': model_id,}
    
    # Invoke the model and print the response
    result = aws_client.invoke_model(**params)
    response = json.loads(result['body'].read().decode())
    return(np.array(response['embeddings'][0]))


In [42]:
sample3 = pd.concat([headlines.head(5), headlines.tail(5)]).copy()
sample3.reset_index(drop=True, inplace=True)

In [43]:
sample3['embedding'] = sample3['headline'].apply(generate_cohere_vector_embedding)

sample3

,Unnamed: 0,headline,url,publisher,date,stock,embedding
0,2,Agilent Technologies Announces Pricing of $5……...,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01 00:00:00,A,"[0.0024700165, -0.053375244, 0.022827148, -0.0..."
1,3,Agilent (A) Gears Up for Q2 Earnings: What's i...,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18 00:00:00,A,"[-0.064697266, -0.029464722, -0.015808105, 0.0..."
2,4,J.P. Morgan Asset Management Announces Liquida...,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15 00:00:00,A,"[-0.012016296, -0.01184082, -0.02633667, 0.002..."
3,5,"Pershing Square Capital Management, L.P. Buys ...",http://www.gurufocus.com/news/1138704/pershing...,GuruFocus,2020-05-15 00:00:00,A,"[-0.019165039, -0.010894775, -0.06549072, -0.0..."
4,6,Agilent Awards Trilogy Sciences with a Golden ...,http://www.gurufocus.com/news/1134012/agilent-...,GuruFocus,2020-05-12 00:00:00,A,"[-0.018173218, -0.009857178, -0.047607422, -0...."
5,1849874,Consumer Cyclical Sector Wrap,https://www.benzinga.com/content/12/08/2846030...,webmaster,2012-08-20 00:00:00,ZX,"[0.014694214, 0.013000488, -0.06890869, -0.072..."
6,1849875,Consumer Cyclical Sector Wrap,https://www.benzinga.com/content/12/07/2767124...,webmaster,2012-07-23 00:00:00,ZX,"[0.014694214, 0.013000488, -0.06890869, -0.072..."
7,1849876,Zacks #5 Rank Additions for Monday - Tale of t...,http://www.zacks.com/stock/news/73497/here-are...,Zacks,2012-04-23 00:00:00,ZX,"[0.025924683, -0.019729614, 0.0035247803, -0.0..."
8,1849877,4 Stock Strategies From Wall Street: Feb. 9 (U...,http://www.thestreet.com/story/11409053/1/4-st...,TheStreet.Com,2012-02-09 00:00:00,ZX,"[0.014862061, -0.037200928, 0.0046081543, 0.02..."
9,1849878,4 Stock Strategies From Wall Street: Feb. 9,https://www.benzinga.com/content/thestreet-com...,webmaster,2012-02-09 00:00:00,ZX,"[0.018096924, -0.037231445, -0.0040740967, 0.0..."


In [45]:
# Test headline
test_headline = "Stock markets, Tariff War"

# Generate embedding for the test
query_vec = generate_cohere_vector_embedding(test_headline)

In [46]:
sample2['cosine_score'] = sample2['embedding'].apply(lambda x: cosine_similarity(x, query_vec))

print(sample2[['headline', 'cosine_score']].sort_values(by='cosine_score', ascending=False))

                                            headline  cosine_score
4  Agilent Awards Trilogy Sciences with a Golden ...      0.093826
5                      Consumer Cyclical Sector Wrap      0.071095
6                      Consumer Cyclical Sector Wrap      0.071095
1  Agilent (A) Gears Up for Q2 Earnings: What's i...      0.048501
3  Pershing Square Capital Management, L.P. Buys ...      0.017611
7  Zacks #5 Rank Additions for Monday - Tale of t...      0.003401
8  4 Stock Strategies From Wall Street: Feb. 9 (U...      0.003111
9        4 Stock Strategies From Wall Street: Feb. 9      0.001565
0  Agilent Technologies Announces Pricing of $5……...      0.000072
2  J.P. Morgan Asset Management Announces Liquida...     -0.044890


Cohere has the overall lowest cosine similarity out of the three, and spaCy has the highest. Like the lesson said, there isn't really a best model just depends on what I'm using it for. spaCy is fast and good for prototyping. Cohere is used for semantic searching, so like when searching on Google "Minecraft movie cast" and on your next search you type "where" and the suggestions might be "where is jack black from" Titan is used when you want to deploy and scale to more people.